# Remote Work and Wage Inequality: Correcting Bias in Regression with Generated Binary Labels

This notebook estimates the association between working from home and salaries using real-world job postings data [(Hansen et al., 2023)](https://dx.doi.org/10.2139/ssrn.4380734). It illustrates how the functions `ols_bca`, `ols_bcm` and `one_step` can be used to correct bias from regressing on AI/ML-generated labels. The notebook reproduces results from Table 1 of [Battaglia, Christensen, Hansen & Sacher (2024)](https://arxiv.org/abs/2402.15585).

In [1]:
%pip install ValidMLInference

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.4/172.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 3.7 MB/s eta 0:00:00


In [2]:
from ValidMLInference import ols, ols_bcm, remote_work_data, one_step_gaussian_mixture
import numpy as np

## Data

The package contains a subset of [a larger dataset](https://wfhmap.com/) regarding work from home. The sample consists of 16,315 job postings for 2022 and 2023 with “San Diego, CA” recorded as the city and “72” recorded as the NAICS2 industry code of the advertising firm.

The data set contains the following entries:
1. `city_name`
2. `naics_2022_2` - an industry code
3. `salary`
4. `wfh_wham` - ML-generated indicator of whether the job offers work from home using fine-tuned DistilBERT as in [(Hansen et al., 2023)](https://dx.doi.org/10.2139/ssrn.4380734)
5. `soc_2021_2` - Bureau of Labor Statistics Standard Occupational Classification code
6. `employment_type_name` - indicates whether the position is full-time or part-time

In [3]:
SD_data = remote_work_data()
SD_data.head(5)

,Unnamed: 0,city_name,naics_2022_2,salary,wfh_wham,soc_2021_2,employment_type_name
0,1,"San Diego, CA",72,57500.0,0,11-0000,Full-time (> 32 hours)
1,2,"San Diego, CA",72,31200.0,0,35-0000,Full-time (> 32 hours)
2,3,"San Diego, CA",72,33280.0,0,35-0000,Part-time / full-time
3,4,"San Diego, CA",72,40560.0,0,35-0000,Full-time (> 32 hours)
4,5,"San Diego, CA",72,45760.0,0,11-0000,Full-time (> 32 hours)


For purpose of this estimation, we also log-transform the salary data.

In [4]:
SD_data['salary'] = np.log(SD_data['salary'])

## Estimating the false-positive rate

The variable `wfh_wham` describing whether the job posting offers remote work is not manually collected, but is imputed via ML methods using fine-tuned DistilBERT as in [(Hansen et al., 2023)](https://dx.doi.org/10.2139/ssrn.4380734). This classifier has over 99% test accuracy in the full sample. Nevertheless, as [Battaglia, Christensen, Hansen & Sacher (2024)](https://arxiv.org/abs/2402.15585) document, even high-performance classifiers can lead to large biases in OLS estimates.

The bias correction methods `ols_bca` and `ols_bcm` require estimates of the classifier's false-positive rate.

We estimate the false positive rate manually. To do so, we took a random sample of size 1000 postings. Of these, 26 had `wfh_wham = 1`. Based on reading these 26 postings, 9 appeared to be misclassified. This means the estimated false-positive rate is 0.009. Accordingly, we will implement `ols_bcm` with `fpr = 0.009` (the estimated false-positive rate) and `m = 1000` (the sample size used to estimate the false-positive rate).

## Results

We first present results for a simple regression of log salary onto the remote work indicator. We then consider a second specification with fixed effects.

We compare standard OLS estimates and confidence intervals with estimates and confidence intervals using `ols_bcm` which performs a direct bias correction and computes bias corrected CIs, and `one_step` which performs maximum likelihood estimation treating the true labels as latent.

### Without fixed effects

We first present OLS estimates:

In [5]:
res = ols(formula = "salary ~ wfh_wham", data=SD_data, intercept = True)
print(res.summary())

            Estimate  Std. Error      z value  P>|z|       2.5%      97.5%
Intercept  10.655967    0.002589  4115.094402    0.0  10.650891  10.661042
wfh_wham    0.648514    0.024911    26.033376    0.0   0.599690   0.697339


Now using the multiplicative bias correction, with bias corrected CIs:

In [7]:
res = ols_bcm(formula= "salary ~ wfh_wham", data=SD_data, fpr = 0.009, m = 1000, intercept=True)
print(res.summary())

            Estimate  Std. Error      z value         P>|z|       2.5%  \
Intercept  10.646261    0.004174  2550.611816  0.000000e+00  10.638080   
wfh_wham    1.052442    0.140035     7.515553  5.662137e-14   0.777978   

               97.5%  
Intercept  10.654442  
wfh_wham    1.326906  


Finally, using maximum likelihood. Since the above residual plots indicate departures from normality, we use a three-component Gaussian mixture to flexibly model the error distribution:

In [8]:
res = one_step_gaussian_mixture(formula = "salary ~ wfh_wham", data = SD_data, generated_var = "wfh_wham", k = 3, nguess = 30, maxiter=300, seed = 123)
print(res.summary())

            Estimate  Std. Error      z value  P>|z|       2.5%      97.5%
Intercept  10.592105    0.002067  5124.310059    0.0  10.588054  10.596156
wfh_wham    0.552343    0.017917    30.828327    0.0   0.517227   0.587460


### With fixed effects

We repeat the above now with fixed effects, which are easily generated for the categorical variables `soc_2021_2` and `employment_type_name`.

First using OLS:

In [9]:
res = ols(formula = "salary ~ wfh_wham + C(soc_2021_2) + C(employment_type_name)", data = SD_data, intercept=True)
summary = res.summary()
rows = ["Intercept", "wfh_wham"]
print(summary.loc[rows])

            Estimate  Std. Error      z value  P>|z|       2.5%      97.5%
Intercept  11.088260    0.009039  1226.680057    0.0  11.070543  11.105977
wfh_wham    0.363921    0.021544    16.892263    0.0   0.321696   0.406146


Now using the multiplicative bias correction:

In [10]:
res = ols_bcm(formula = "salary ~ wfh_wham + C(soc_2021_2) + C(employment_type_name)", generated_var = "wfh_wham", data = SD_data, fpr = 0.009, m=1000)
summary = res.summary()
print(summary.loc[rows])

            Estimate  Std. Error      z value         P>|z|       2.5%  \
Intercept  11.074074    0.010335  1071.556274  0.000000e+00  11.053818   
wfh_wham    0.641276    0.099605     6.438210  1.208906e-10   0.446054   

               97.5%  
Intercept  11.094329  
wfh_wham    0.836497  


Comparing these results with the OLS results above, we see that the bias corrected CI for the slope coefficient lies to the right of the OLS CI.

Finally, using maximum likelihood:

In [ ]:
res = one_step_gaussian_mixture(formula = "salary ~ wfh_wham + C(soc_2021_2) + C(employment_type_name)", data = SD_data, generated_var = "wfh_wham", k = 3, nguess = 30, maxiter=300)
summary = res.summary()
print(summary.loc[rows])

            Estimate  Std. Error      z value  P>|z|       2.5%      97.5%
Intercept  10.890429    0.007684  1417.218872    0.0  10.875367  10.905490
wfh_wham    0.518930    0.015346    33.815811    0.0   0.488853   0.549007
